# Cytof data processing

## Requirements

For compatibility, the `numpy` version needs to be fixed:
```bash
pip3 install pandas numpy scprep statsmodels seaborn
```

## Variables definition

In [1]:
# Specify the folder containing the CSV files and the output file name
folder_path = '/Users/romain/Documents/Perso/aure/samples'

#This is a single file containing all conditions. Files have been downloaded from Cytobank as txt files, then concatenated together using R.
normalisation_channel = '165Ho_AlexaFluor488'

# Specify the metadata columns
# metadata_columns = ['Cell_Index', 'Condition', 'Control', 'Replicate', normalisation_channel]
metadata_columns = ['Cell_Index', normalisation_channel]

# Specify other columns to exclude from processing
excluded_columns = []

# Compute the non data columns in a new variable for easier later use
non_data_columns = excluded_columns + metadata_columns

## Import common packages

In [2]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 1000)

## Concatenate all files

### Configuration

In [3]:
get_condition_replicate_from_filename = True

### Concatenate

In [ ]:
all_events = pd.DataFrame()

# Loop over all files in folder
for filename in os.listdir(folder_path):
    # Only consider files with '.txt' extension
    if filename.endswith('.txt'):
        # Build the full path to file
        file_path = os.path.join(folder_path, filename)
        # Load the file
        events = pd.read_csv(file_path, delimiter='\t')

        if get_condition_replicate_from_filename:
            # Retrieve metadata from the filename (ex: WGANormalised_Pro_PDO21 + CAFs_01.fcs_file_internal...)
            # First split: ['Pro_AD022_AD_PDO21_OldMG_DMSO_02', '_file_internal']
            # Second split over first element: ['Pro', 'AD022', 'AD', 'PDO21+CAFs', 'OldMG', 'DMSO', '02']

            # Pro_AD022_AD_PDO21+CAFs_OldMG_DMSO_02.fcs_file_internal_comp_PDOs
            metadata_from_filename = filename.split('.fcs')[0].split('_')
            replicate = metadata_from_filename[-1] # 02
            drug = metadata_from_filename[-2] # DMSO
            treatment = metadata_from_filename[-3] # OldMG
            culture = metadata_from_filename[-4] # PDO21+CAFs
            events['Culture'] = culture
            events['Treatment'] = treatment
            # Store the control name in the dataframe: second-to-last element split over '+', and stripped to remove whitespace from both sides
            # events['Control'] = culture.split('+')[0].strip()
            # Store the replicate in the dataframe: last element
            events['Replicate'] = replicate # 02
            events['Drug'] = drug # suposed to be DMSO
            # events['Condition'] = culture + '_' + treatment + '_' + replicate
            metadata_columns += ['Culture', 'Treatment', 'Replicate', 'Drug']


        # Add the file data to the DataFrame containing all events
        all_events = pd.concat([all_events, events], ignore_index=True)

metadata_columns = list(set(metadata_columns))
# Print all events
initial_all_events = all_events.copy()
all_events

## Data correction

### Loess

In [5]:
import statsmodels.api as sm

# Advantages of Loess-Based Correction
#
# ✅ Handles Non-Linearity: No assumption about the functional form of the relationship.
# ✅ Data-Driven Approach: Adapts to trends within the dataset.
# ✅ Preserves Biological Signal: Corrects only for size effects without distorting underlying biological differences.
#
# Challenges & Considerations
#
# ⚠ Computational Cost: Loess smoothing can be slow for large datasets.
# ⚠ Choosing the Smoothing Parameter (span or frac): Needs tuning to avoid overfitting or underfitting.
# ⚠ Boundary Effects: Can be unstable at extreme values of WGA.
def apply_loess_correction(maker_data, wga_data):
    # Fit Loess model
    lowess_fit = sm.nonparametric.lowess(maker_data, wga_data, frac=0.5)

    # Interpolating fitted values
    # - Column 0 (lowess_fit[:, 0]): The independent variable values (e.g., WGA, the size marker). | WGA values (sorted).
	# - Column 1 (lowess_fit[:, 1]): The corresponding smoothed (Loess-fitted) dependent variable values (e.g., marker intensity). | Loess-smoothed marker intensity values.
    fitted_values = np.interp(wga_data, lowess_fit[:, 0], lowess_fit[:, 1])

    # Return the corrected data
    return maker_data / fitted_values

In [ ]:
from joblib import Parallel, delayed

# Separate the columns to normalize and the columns to leave as-is
columns_to_normalize = all_events.columns.difference(metadata_columns).tolist()

norm_channel_data = all_events[normalisation_channel]

# Apply Loess correction to each marker column
# Use joblib to parallelize
results = Parallel(n_jobs=-1, backend='loky')(  
    # n_jobs=-1: Use all available cores
    # backend='loky': Loky backend for robust parallelization
    delayed(apply_loess_correction)(all_events[marker], norm_channel_data)
    for marker in columns_to_normalize
)

# Filter to keep only rows with all finite values
# all_events = all_events[np.isfinite(all_events).all(axis=1)]

corrected_events = pd.concat(results, axis=1)

## EMD Generation

### Prepare the data

In [7]:
all_events = initial_all_events.copy()

In [ ]:
#drop the metadata to create a df with only numerical data for normalisation/transformation
data = all_events.drop(metadata_columns ,axis=1)
data

In [ ]:
metadata_columns

In [ ]:
#make sure all metadata columns are strings (not numberical as this will run into errors)
metadata = all_events.filter(metadata_columns)
metadata[metadata_columns] = metadata[metadata_columns].applymap(str)
metadata  

### Select a subset of data (optional)

In [11]:
#Batches:
#Batch 1 = PDO27wt/ko exp B BM/MOPC21/B7C18
#Batch 2 = PDO27 ABCEDF7 Tr
#Batch 3 = PDO27 ABCDEF7 NT
#Batch 4 = PDO21/23/216 ABE7 Tr
#Batch 5 = PDO21/23/216 ABE7 NT 
#Batch 6 = PDO5/11 ABE7 Tr/NT
#Batch 7 = PDO75/99 ABE7 Tr/NT
#Batch 8 = PDO109/141 ABE7 Tr/NT
#Batch 9 = NT/eGFP/eGFP-stIL15 ABE7

#### Configuration

In [12]:
# To enable this process, set this variable to True, False otherwise
should_select_a_subset = False

subset_condition = [True]
if should_select_a_subset:
    # Define here the filter to apply
    subset_condition = \
        metadata['Patient'].isin(['X','5','11','21','23','27','75','99','109','141','216']) & \
        metadata['gd_donor'].isin(['A','B','E','7']) & \
        metadata['Transduction'].isin(['eGFP-stIL15']) & \
        metadata['Treatment'].isin(['BM','B7C18']) & \
        metadata['Batch'].isin(['Batch2','Batch4','Batch6','Batch7','Batch8'])


#### Select the data

In [13]:
if should_select_a_subset:
    #Select eGFP-stIL15 / ABE7 / wt PDO / BM / B7C18 (I was just selecting the data I wanted to use)
    data = data.loc[subset_condition]
    data

#### Select the metadata

In [14]:
if should_select_a_subset:
    #selecting the corresponding metadata
    metadata = metadata.loc[subset_condition]
    metadata

### Arcsinh transformation

#### Configuration

In [15]:
arcsinh_cofactor = 5

#### Data processing

In [ ]:
#arcsinh transformation of all raw data
data = np.arcsinh(data/arcsinh_cofactor)
data

### Batch effect correction

In [17]:
import scprep.normalize

# Data centering by batch to correct any cytof batch effect
# Only if 'Batch' is a metadata
if 'Batch' in metadata.columns:
    data = scprep.normalize.batch_mean_center(data,sample_idx=metadata['Batch'])
    data

### Re-assemble processed data with metadata

#### Concatenate data with metadata

In [ ]:
# Combine arcsinh-transformed and mean-centered data with metadata again
processed_data = pd.concat([data, metadata], axis=1)
processed_data

#### Re-index the Dataframe

In [19]:
row_count = processed_data.shape[0]
processed_data.index = np.arange(row_count)

#### Ensure type of metadata column to be string

In [ ]:

processed_data[metadata_columns] = processed_data[metadata_columns].applymap(str)
processed_data

### Store the `Condition` information (optional)

#### Configuration

In [21]:
condition_colmns = ['Culture', 'Treatment', 'Drug', 'Replicate']

#### Generate the `Condition` column

In [ ]:
if 'Condition' not in metadata.columns:
    # Create a condition column for every cell in the experiment
    processed_data['Condition'] = processed_data[condition_colmns].astype(str).agg('_'.join, axis=1)

    # Add `Condition` to the list of metadata columns
    metadata_columns += ['Condition']

processed_data

### Store the `Control` information (optional)

#### Configuration

In [36]:
control_culture = 'PDO21'
control_columns = ['Treatment']
control_drug = 'DMSO'

#### Generate the `Control` column

In [ ]:
if 'Control' not in metadata.columns:
    # Define control for pairwise EMD. 
    # processed_data['Control'] = processed_data[control_columns].astype(str).agg('_'.join, axis=1)
    processed_data['Control'] = control_culture + "_" + processed_data[control_columns].astype(str).agg('_'.join, axis=1) + "_" + control_drug

    # Add `Control` to the list of metadata columns
    metadata_columns += ['Control']

processed_data

### Save processed data to file (optional)

In [38]:
processed_data.to_csv(folder_path + '/processed_data.bkp', index=False, sep='\t')

### Initialise EMD dataframe

#### Compute the markers list

In [ ]:
# For each column in the Dataframe, keep only the ones not in the `metadata_columns` variable
markers_list = [col for col in processed_data.columns if col not in metadata_columns]
# marker_list = list(processed_data.columns.values)
markers_list

#### Compute the conditions list

In [ ]:
# Get the list of unique conditions
conditions_list = pd.unique(processed_data['Condition'].tolist())
conditions_list

# pdo21 et pdo21+caf

#### Compute the controls list (unused)

In [ ]:
# Get the list of unique controls
controls_list = pd.unique(processed_data['Control'].tolist())
controls_list

#### Create the DataFrame that will receive the EMD values

In [42]:
# Empty df with NaN values to populate with the EMD values
emd_dataframe = pd.DataFrame(
    np.full(
        (len(conditions_list), len(markers_list)), 
        np.nan),
    columns = markers_list,
    index = conditions_list)


### Calculate EMD scores

In [43]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import scprep.stats

# Precompute control dataframes for each control name
control_data = {
    control: processed_data.loc[processed_data["Condition"].str.startswith(control)].copy()
    for control in processed_data['Control'].unique()
}

# Function to process a single (condition, marker) pair
def process_condition_marker(condition, marker):
    # Dataframe of all events for the condition in the list
    condition_events = processed_data.loc[processed_data["Condition"] == condition].copy()
    control_name = condition_events['Control'].values[0]
    # Dataframe of all events from the control that will be compared with the events of the current condition
    control_df = control_data[control_name].copy()
    
    condition_median = condition_events[marker].median()
    control_median = control_df[marker].median()

    # Check the sign by using the `median` values
    sign = np.sign(condition_median - control_median)
    
    # Fall back to mean if medians are equal
    if sign == 0:
        sign = np.sign(
            condition_events[marker].mean() - control_df[marker].mean()
        )
    
    # Compute the EMD
    emd = scprep.stats.EMD(condition_events[marker], control_df[marker])
    return (condition, marker, sign * emd)


# Use joblib to parallelize
results = Parallel(n_jobs=-1, backend='loky')(  
    # n_jobs=-1: Use all available cores
    # backend='loky': Loky backend for robust parallelization
    delayed(process_condition_marker)(condition, marker)
    for condition in conditions_list
    for marker in markers_list
)

# Convert results into a DataFrame
emd_dataframe = (
    pd.DataFrame(results, columns=['Condition', 'Marker', 'Signed EMD'])
    .pivot(index='Condition', columns='Marker', values='Signed EMD')
)


In [ ]:
emd_dataframe.T

### Save EMD to file

In [ ]:
emd_dataframe.to_csv(folder_path + '/events_emd.bkp', index=True, sep='\t')
emd_dataframe

## Generate Heatmap

### Configure

In [46]:
# import numpy as np
# import pandas as pd
# import plotly.express as px
import matplotlib.pyplot as plt
# %matplotlib inline

# import phate
# import matplotlib
# import matplotlib.colors as mcolors
# import scprep
# import os
# import scipy
import seaborn as sns
# from scipy.stats.stats import pearsonr
# from sklearn.decomposition import PCA

# generate custom colour palette for seaborn heatmaps

import matplotlib.colors as mcolors

# Define custom colormap with white at zero
custom_diverging = mcolors.LinearSegmentedColormap.from_list("",["#6f00ff", "white", "#ef3038"], N=256)

### Draw

In [ ]:
fig, ax = plt.subplots(figsize=(10, 12))

offset = mcolors.TwoSlopeNorm(vmin=-1 ,vcenter=0, vmax=1)

sns.heatmap(emd_dataframe.T, cmap=custom_diverging, norm=offset, )

plt.savefig(folder_path + '/emd_all_global.png', dpi=900, transparent=True)
plt.show()